In [1]:
import sys
sys.path.insert(0, "..")
from pathlib import Path
import re
import json
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.collections as mc
import matplotlib.cm as cm
from matplotlib.container import BarContainer
from matplotlib.ticker import FuncFormatter

def comma_formatter(x, pos):
    return f'{x:,.0f}'

def pretty_fmt(num, suffix="B"):
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if abs(num) < 1024.0:
            return f"{num:.0f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Y{suffix}"

BENCHMARK_NAME="rdbench"
FIG_DIR = Path(f"../../fig/{BENCHMARK_NAME}").resolve()
FIG_DIR.mkdir(parents=True, exist_ok=True)
def save(fig, name):
    for ext in ('png', 'eps'):
        fig.savefig(FIG_DIR / f'{name}.{ext}', bbox_inches = 'tight')
    # pass 


In [2]:
re_hook_pmembb = re.compile(r"hook:pmembb:rpmbb_store_(create|free|save|load) ([0-9]+\.[0-9]+)")
def df_extract_hook_pmembb_time(stderr_path: Path):
    text = stderr_path.read_text()
    matches = re_hook_pmembb.findall(text)
    return pd.DataFrame(matches, columns=["type", "time"])

def extract_hook_pmembb_time_max(stderr_path: Path):
    return (
        df_extract_hook_pmembb_time(stderr_path)
        .groupby("type")
        .max()
        .transpose()
        .rename(columns={
            "create": "createTime",
            "free": "freeTime",
            "save": "saveTime",
            "load": "loadTime",
        })
        .reset_index(drop=True)
        .astype(float)
        .loc[0,:]
        .to_dict()
    )

wpath = Path("../../raw/ior-pmembb/2024.01.23-15.22.18-default/2024.01.23-15.24.03-166930.nqsv-1/ior_stderr_8.txt")
rpath = Path("../../raw/ior-pmembb/2024.01.23-15.22.18-default/2024.01.23-15.24.03-166930.nqsv-1/ior_stderr_8.txt")
extract_hook_pmembb_time_max(rpath)


{'createTime': 14.976, 'freeTime': 0.023, 'loadTime': 0.005}

In [3]:
RAW_BASEDIR=Path(f"../../raw/")

default_metadata={
}

job_group = (
    ("rdbench-pmembb/2024.01.29-15.40.22-nosync", {"storageSystem": "PMEMBB"}),
    # ("rdbench-beeond/2024.02.19-16.31.53-default", {}),
    ("rdbench-lustre/2024.02.21-14.28.22-default", {}),
    ("rdbench-chfs/2024.02.22-23.25.04-default", {}),
    ("rdbench-chfs/2024.02.22-23.25.54-default", {}),
    ("rdbench-unifyfs/2024.02.25-18.09.21-default", {}),
    ("rdbench-unifyfs/2024.02.25-18.37.21-nnodes36-49", {}),
    ("rdbench-beeond/2024.02.28-07.02.06-setpattern", {}),
)

def job_dirs(job_group_dir: str):
    dir = RAW_BASEDIR / job_group_dir
    return (f.resolve() for f in dir.iterdir() if f.is_dir())

re_timestamp = re.compile(r'([0-9]{4})\.([0-9]{2})\.([0-9]{2})-([0-9]{2})\.([0-9]{2})\.([0-9]{2})')
def parse_job_start_timestamp(job_dir: Path):
    m = re_timestamp.search(str(job_dir))
    if m:
        return m.group()
    else:
        return None

re_runid = re.compile(r"_([0-9]+)\.json")
def get_runid(file: Path):
    m = re_runid.search(str(file))
    if m:
        return int(m.group(1))
    else:
        return None

def load_nodelist(job_dir: Path):
    with(job_dir / f"nodelist").open() as f:
        return list(map(lambda s: s.strip(), f.readlines()))

src = []
for job_group_dir,metadata in job_group:
    for job_dir in job_dirs(job_group_dir):
        try:
            job_start = {"job_start": parse_job_start_timestamp(job_dir)}
            job_nodelist = {"nodelist": load_nodelist(job_dir)}
        except Exception as e:
            print(f"{job_dir}, {e}")
        for job_params_file in job_dir.glob("job_params_*.json"):
            try:
                runid = get_runid(job_params_file)
                job_params = json.loads(job_params_file.read_text())
                rdbench_result_file = job_dir / f"rdbench_stdout_{runid}.json"
                rdbench_result = json.loads(rdbench_result_file.read_text())
                pmembb_time = extract_hook_pmembb_time_max(job_dir / f"rdbench_stderr_{runid}.txt")
                src.append({
                    **default_metadata,
                    **metadata,
                    **job_start,
                    **job_nodelist,
                    **job_params,
                    **pmembb_time,
                    **rdbench_result,
                    "runid": runid,
                })
            except FileNotFoundError as e:
                print(e)
            except Exception as e:
                print(f'{job_dir}/{job_params_file}, {e}')

# df_src = (
#     pd.DataFrame(src)
#     .assign(odirect=lambda x: x["Command line"].str.contains("--posix.odirect"))
#     .assign(strided=lambda x: x["summary_transferSize"] == x["summary_blockSize"])
#     .rename({
#         "summary_bwMaxMIB": "bw",
#         "summary_MeanTime": "time",
#         "summary_segmentCount": "segmentCount",
#         "summary_blockSize": "blockSize",
#         "summary_transferSize": "transferSize",
#         "summary_operation": "operation",
#         "Parameters_filePerProc": "fpp",
#         "Parameters_reorderTasks": "reorder",
#         "summary_API": "api",
#         "result_access": "access",
#         "result_bwMiB": "bwMiB",
#         "result_openTime": "openTime",
#         "result_wrRdTime": "wrRdTime",
#         "result_closeTime": "closeTime",
#         "result_totalTime": "totalTime",
#     }, axis=1)
#     .astype({"fpp": bool, "reorder": bool,})
#     .assign(bwGB=lambda x: x["bwMiB"] * 2**20 / 10**9)
# )
# df_src["label"] = np.select([
#     (df_src["access"] == "write"),
#     (df_src["access"] == "read") & (df_src["reorder"] == True),
#     (df_src["access"] == "read") & (df_src["reorder"] == False),
# ], ["Write", "Read (remote)", "Read (local)"])


df_src = (
    pd.DataFrame(src)
    .drop(columns=[
        'phaseDurationsSec',
    ])
    .assign(bwGB=lambda x: x['writeBandwidthByte'] / 1000**3)
)

df_src.head()


storageSystem            job_start  \
0        PMEMBB  2024.01.29-15.40.22   
1        PMEMBB  2024.01.29-15.40.22   
2        PMEMBB  2024.01.29-15.40.22   
3        PMEMBB  2024.01.29-15.40.22   
4        PMEMBB  2024.01.29-15.40.22   

                                            nodelist  nnodes  ppn    np  \
0  [bnode068, bnode069, bnode070, bnode071, bnode...       9   48   432   
1  [bnode071, bnode072, bnode073, bnode074, bnode...      16   48   768   
2  [bnode030, bnode031, bnode032, bnode040, bnode...      36   48  1728   
3           [bnode026, bnode027, bnode028, bnode029]       4   48   192   
4  [bnode022, bnode023, bnode024, bnode025, bnode...      49   48  2352   

         jobid  runid    pmem_path  pmem_size  ...  \
0  168089.nqsv      0  /dev/dax0.0        0.0  ...   
1  168095.nqsv      0  /dev/dax0.0        0.0  ...   
2  168102.nqsv      0  /dev/dax0.0        0.0  ...   
3  168084.nqsv      0  /dev/dax0.0        0.0  ...   
4  168104.nqsv      0  /dev/dax0.0        0.0  ...   

  UNIFYFS_MARGO_SERVER_TIMEOUT  UNIFYFS_MARGO_SERVER_POOL_SIZE  \
0                          NaN                             NaN   
1                          NaN                             NaN   
2                          NaN                             NaN   
3                          NaN                             NaN   
4                          NaN                             NaN   

   UNIFYFS_MARGO_CLIENT_POOL_SIZE UNIFYFS_RUNSTATE_DIR  \
0                             NaN                  NaN   
1                             NaN                  NaN   
2                             NaN                  NaN   
3                             NaN                  NaN   
4                             NaN                  NaN   

   UNIFYFS_SERVER_MAX_APP_CLIENTS  UNIFYFS_SERVER_INIT_TIMEOUT  \
0                             NaN                          NaN   
1                             NaN                          NaN   
2                             NaN                          NaN   
3                             NaN                          NaN   
4                             NaN                          NaN   

   UNIFYFS_SERVER_LOCAL_EXTENTS UNIFYFS_CLIENT_LOCAL_EXTENTS  \
0                           NaN                          NaN   
1                           NaN                          NaN   
2                           NaN                          NaN   
3                           NaN                          NaN   
4                           NaN                          NaN   

   UNIFYFS_CLIENT_NODE_LOCAL_EXTENTS        bwGB  
0                                NaN  197.647020  
1                                NaN  339.348800  
2                                NaN  736.188194  
3                                NaN   87.715552  
4                                NaN  893.625829  

[5 rows x 69 columns]

In [4]:
df = df_src

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# display(df.columns.to_list())

(
    df[[
        "storageSystem",
        "spack_env_name",
        "nnodes",
        "ppn",
        "bwGB",
    ]]
    # .head()
)


storageSystem              spack_env_name  nnodes  ppn         bwGB
0         PMEMBB  mpiio-pmembb-deferred-open       9   48   197.647020
1         PMEMBB  mpiio-pmembb-deferred-open      16   48   339.348800
2         PMEMBB                mpiio-pmembb      36   48   736.188194
3         PMEMBB                mpiio-pmembb       4   48    87.715552
4         PMEMBB                mpiio-pmembb      49   48   893.625829
5         PMEMBB  mpiio-pmembb-deferred-open      36   48   706.817777
6         PMEMBB                mpiio-pmembb      49   48   959.948339
7         PMEMBB                mpiio-pmembb      25   48   520.826046
8         PMEMBB                mpiio-pmembb       9   48   192.837172
9         PMEMBB  mpiio-pmembb-deferred-open       1   48    23.676850
10        PMEMBB  mpiio-pmembb-deferred-open      25   48   511.769965
11        PMEMBB  mpiio-pmembb-deferred-open       4   48    92.500124
12        PMEMBB                mpiio-pmembb      64   48  1168.441238
13        PMEMBB  mpiio-pmembb-deferred-open       1   48    23.256679
14        PMEMBB  mpiio-pmembb-deferred-open      16   48   345.356496
15        PMEMBB                mpiio-pmembb      36   48   731.825098
16        PMEMBB  mpiio-pmembb-deferred-open      25   48   501.785141
17        PMEMBB  mpiio-pmembb-deferred-open       9   48   198.569244
18        PMEMBB  mpiio-pmembb-deferred-open      49   48   861.852578
19        PMEMBB                mpiio-pmembb       9   48   192.825645
20        PMEMBB  mpiio-pmembb-deferred-open      49   48   926.461873
21        PMEMBB  mpiio-pmembb-deferred-open      64   48  1132.575408
22        PMEMBB                mpiio-pmembb       1   48    23.455013
23        PMEMBB  mpiio-pmembb-deferred-open      64   48  1103.852057
24        PMEMBB  mpiio-pmembb-deferred-open      36   48   722.689719
25        PMEMBB                mpiio-pmembb      25   48   515.568929
26        PMEMBB                mpiio-pmembb       4   48    87.633009
27        PMEMBB                mpiio-pmembb      64   48  1221.952894
28        PMEMBB                mpiio-pmembb       1   48    23.570439
29        PMEMBB                mpiio-pmembb      16   48   336.602866
30        PMEMBB  mpiio-pmembb-deferred-open       4   48    87.712020
31        PMEMBB                mpiio-pmembb      16   48   337.448248
32        Lustre                mpiio-pmembb      36   48     6.218528
33        Lustre                mpiio-pmembb      36   36    11.184768
34        Lustre                mpiio-pmembb      16   48     7.073448
35        Lustre                mpiio-pmembb      16   36    10.778269
36        Lustre                mpiio-pmembb      49   48     3.243354
37        Lustre                mpiio-pmembb      49   36     7.902168
38        Lustre                mpiio-pmembb       9   48     7.462358
39        Lustre                mpiio-pmembb       9   36     9.078811
40        Lustre                mpiio-pmembb       4   48     5.235540
41        Lustre                mpiio-pmembb       4   36     6.064278
42        Lustre                mpiio-pmembb      64   48     7.869038
43        Lustre                mpiio-pmembb      64   36     2.136745
44        Lustre                mpiio-pmembb       1   48     2.070287
45        Lustre                mpiio-pmembb       1   36     2.363895
46        Lustre                mpiio-pmembb      25   48     6.636513
47        Lustre                mpiio-pmembb      25   36    11.706497
48          CHFS                        chfs       1   36     3.780498
49          CHFS                        chfs       4   36    15.696539
50          CHFS                        chfs      25   36    78.459459
51          CHFS                        chfs       4   36    14.996027
52          CHFS                        chfs       9   36    33.952820
53          CHFS                        chfs       9   36    32.450996
54          CHFS                        chfs      16   36    54.998135
55          CHFS           

In [5]:
df = df_src
same_setting_columns = [
    "storageSystem",
    "spack_env_name",
    "nnodes",
    "ppn",
    # "lustre_stripe_count",
]
same_setting_grouper = df.groupby(same_setting_columns)

(
    same_setting_grouper
    .size()
    .plot(figsize=(32, 9), kind="bar")
)


<Axes: xlabel='storageSystem,spack_env_name,nnodes,ppn'>

In [6]:
df = (
    same_setting_grouper["bwGB"].max()
    # .agg({
    #     "bw": "max",
    #     "time": "min",
    # })
    .reset_index()
    # .assign(bwGB=lambda x: x["bw"] * 2**20 / 10**9)
    # .query("lustre_stripe_count == 1")
    # .query("spack_env_name == 'mpiio-pmembb'")
    .query("spack_env_name != 'mpiio-pmembb-deferred-open'")
    .query(
        "(storageSystem == 'Lustre' and ppn == 36) or "
        "(storageSystem == 'BeeOND' and ppn == 36) or "
        "(storageSystem == 'CHFS' and ppn == 36) or "
        "(storageSystem == 'UnifyFS' and ppn == 36) or "
        "(storageSystem == 'PMEMBB' and ppn == 48)"
    )
    # .query("not ('storageSystem' == 'Lustre' and 'ppn' != 48)")
)

display(df)

df_plot = df.pivot(index="nnodes", columns=["storageSystem", "ppn"], values=["bwGB"])
df_plot.columns = df_plot.columns.get_level_values(1)
df_plot = df_plot.loc[:, [
    "Lustre",
    "BeeOND",
    "CHFS",
    "UnifyFS",
    "PMEMBB",
]].rename({
    "PMEMBB": "PEANUTS",
}, axis=1)

display(df_plot)


fig = plt.figure(figsize=(8, 6), dpi=100)
fig.patch.set_alpha(1)

ax = df_plot.plot(
    ax=fig.add_subplot(),
    kind="bar",
    # kind="line",
    # xlabel="Transfer size [B]",
    xlabel="Number of nodes",
    ylabel="Bandwidth [GB/s]",
    # style=".-",
    legend=True,
    rot=0,
)

# ax.set_title(f"RDBench")
# ax.set_xscale("log", base=2)
# ax.set_xticks(df_plot.index, [pretty_fmt(x, suffix="") for x in df_plot.index])
ax.grid(which = "major", axis = "y", alpha = 0.8, linestyle = "--", linewidth = 1)
# handles, labels = ax.get_legend_handles_labels()# 凡例の情報を取得
# ax.legend(title='nnodes', handles=handles[::-1],labels=labels[::-1])# 判例を逆順に表示
ax.yaxis.set_major_formatter(FuncFormatter(comma_formatter))

# legend_title = ax.get_legend().get_title().get_text()
# ax.legend(title=legend_title, bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
ax.legend(title="")
plt.tight_layout()

fig.show()
plt.show()
save(fig, F"rdbench_write")


storageSystem spack_env_name  nnodes  ppn         bwGB
0         BeeOND   mpiio-pmembb       1   36     1.021669
2         BeeOND   mpiio-pmembb       4   36     3.535209
4         BeeOND   mpiio-pmembb       9   36     7.678511
6         BeeOND   mpiio-pmembb      16   36    13.351534
8         BeeOND   mpiio-pmembb      25   36    21.828567
10        BeeOND   mpiio-pmembb      36   36    33.015279
12        BeeOND   mpiio-pmembb      49   36    44.547038
14        BeeOND   mpiio-pmembb      64   36    57.811311
16          CHFS           chfs       1   36     3.830495
17          CHFS           chfs       4   36    15.696539
18          CHFS           chfs       9   36    33.952820
19          CHFS           chfs      16   36    55.194750
20          CHFS           chfs      25   36    78.459459
21          CHFS           chfs      36   36    91.853072
22          CHFS           chfs      49   36   103.107552
23          CHFS           chfs      64   36   143.041560
24        Lustre   mpiio-pmembb       1   36     2.363895
26        Lustre   mpiio-pmembb       4   36     6.064278
28        Lustre   mpiio-pmembb       9   36     9.078811
30        Lustre   mpiio-pmembb      16   36    10.778269
32        Lustre   mpiio-pmembb      25   36    11.706497
34        Lustre   mpiio-pmembb      36   36    11.184768
36        Lustre   mpiio-pmembb      49   36     7.902168
38        Lustre   mpiio-pmembb      64   36     2.136745
40        PMEMBB   mpiio-pmembb       1   48    23.570439
41        PMEMBB   mpiio-pmembb       4   48    87.715552
42        PMEMBB   mpiio-pmembb       9   48   192.837172
43        PMEMBB   mpiio-pmembb      16   48   337.448248
44        PMEMBB   mpiio-pmembb      25   48   520.826046
45        PMEMBB   mpiio-pmembb      36   48   736.188194
46        PMEMBB   mpiio-pmembb      49   48   959.948339
47        PMEMBB   mpiio-pmembb      64   48  1221.952894
56       UnifyFS        unifyfs       1   36    11.038148
57       UnifyFS        unifyfs       4   36    38.651425
58       UnifyFS        unifyfs       9   36    70.237729
59       UnifyFS        unifyfs      16   36   119.027807
60       UnifyFS        unifyfs      25   36   139.043691
61       UnifyFS        unifyfs      36   36   176.982077
62       UnifyFS        unifyfs      49   36   194.592285
63       UnifyFS        unifyfs      64   36   193.464454

storageSystem     Lustre     BeeOND        CHFS     UnifyFS      PEANUTS
nnodes                                                                  
1               2.363895   1.021669    3.830495   11.038148    23.570439
4               6.064278   3.535209   15.696539   38.651425    87.715552
9               9.078811   7.678511   33.952820   70.237729   192.837172
16             10.778269  13.351534   55.194750  119.027807   337.448248
25             11.706497  21.828567   78.459459  139.043691   520.826046
36             11.184768  33.015279   91.853072  176.982077   736.188194
49              7.902168  44.547038  103.107552  194.592285   959.948339
64              2.136745  57.811311  143.041560  193.464454  1221.952894

/tmp/ipykernel_3075/1547427909.py:67: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
